In [64]:
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [65]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/tylerhill/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


In [66]:
url = "https://redplanetscience.com"
browser.visit(url)

html = browser.html

In [68]:
soup = bs(html, 'html.parser')

In [69]:
soup

<html><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
<link href="css/font.css" rel="stylesheet" type="text/css"/>
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet"/>
<title>News - Mars Exploration Program</title>
</head>
<body>
<div class="col-md-12">
<div class="row">
<nav class="navbar navbar-expand-lg navbar-light fixed-top">
<div class="container-fluid">
<a class="navbar-brand" href="#">
<img src="image/nasa.png" width="80"/><span class="logo">MARS Planet Science</span>
<span class="logo1">Exploration Program</spa

In [116]:
results = soup.find_all('div', id='news')
print(results)

[<div class="container" id="news">
<div class="col-md-12">
<hr/>
<div class="row">
<div class="col-md-4">
<div class="list_image">
<img src="https://mars.nasa.gov/system/news_items/list_view_images/8506_mars-solar-conjunction-th.jpg"/>
</div>
</div>
<div class="col-md-8">
<div class="list_text">
<div class="list_date">June 4, 2022</div>
<div class="content_title">What's Mars Solar Conjunction, and Why Does It Matter?</div>
<div class="article_teaser_body">NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.</div>
</div>
</div>
</div>
</div>
<div class="col-md-12">
<hr/>
<div class="row">
<div class="col-md-4">
<div class="list_image">
<img src="https://mars.nasa.gov/system/news_items/list_view_images/8718_25106_PIA23986-320.jpg"/>
</div>
</div>
<div class="col-md-8">
<div class="list_text">
<div class="list_date">June 3, 2022</div>
<div class="content_title">NASA's Mars Perseverance Rover Passes Fli

In [117]:
# results = soup.find_all('div', class_="list_text")

In [127]:
for result in results:
    news_title = soup.find('div', class_='content_title').text
    news_p = soup.find('div', class_='article_teaser_body').text
    print(news_title)
    print('-'*50)
    print(news_p)

What's Mars Solar Conjunction, and Why Does It Matter?
--------------------------------------------------
NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.


In [86]:
# news_title = soup.find('div', class_='content_title')
# news_p = soup.find('div', class_='article_teaser_body')
# queries = result.find_all('li')